<a href="https://colab.research.google.com/github/HUJameson/Colab/blob/main/aillm_0203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers openai faiss-cpu llama-index llama-cpp-python


In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/My Drive/Colab Notebooks")

Mounted at /content/drive


In [3]:
import openai, os
import faiss
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTListIndex, ServiceContext
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from llama_index.node_parser import SimpleNodeParser

openai.api_key = ""

text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=100, chunk_overlap=20)
parser = SimpleNodeParser(text_splitter=text_splitter)
documents = SimpleDirectoryReader('./data/faq/').load_data()
nodes = parser.get_nodes_from_documents(documents)

embed_model = LangchainEmbedding(HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
))
service_context = ServiceContext.from_defaults(embed_model=embed_model)

dimension = 768
faiss_index = faiss.IndexFlatIP(dimension)
#index = GPTListIndex(nodes=nodes,faiss_index=faiss_index, service_context=service_context)
index = GPTListIndex(nodes=nodes, service_context=service_context)

******
Could not load OpenAI model. Using default LlamaCPP=llama2-13b-chat. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

******
total size (MB): 7365.83


7025it [00:56, 124.87it/s]                          
AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 
[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#from llama_index import QueryMode

#openai.api_key = os.environ.get("OPENAI_API_KEY")

query_engine = index.as_query_engine(response_mode='tree_summarize', verbose=True)
response = query_engine.query("请问你们海南能发货吗？")
print(response)


In [ ]:
response = query_engine.query("你们用哪些快递公司送货？")
print(response)

In [ ]:
!pip install icetk cpm_kernels

In [ ]:
!pip install transformers

In [ ]:
!pip install --upgrade protobuf

In [5]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b-int4", trust_remote_code=True)
model = AutoModel.from_pretrained("THUDM/chatglm-6b-int4", trust_remote_code=True).half().cuda()
model = model.eval()

No compiled kernel found.
Compiling kernels : /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/6c5205c47d0d2f7ea2e44715d279e537cae0911f/quantization_kernels.c
Compiling gcc -O3 -fPIC -std=c99 /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/6c5205c47d0d2f7ea2e44715d279e537cae0911f/quantization_kernels.c -shared -o /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/6c5205c47d0d2f7ea2e44715d279e537cae0911f/quantization_kernels.so
Load kernel : /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/6c5205c47d0d2f7ea2e44715d279e537cae0911f/quantization_kernels.so
Using quantization cache
Applying quantization to glm layers


In [6]:
question = """
自收到商品之日起7天内，如产品未使用、包装完好，您可以申请退货。某些特殊商品可能不支持退货，请在购买前查看商品详情页面的退货政策。

根据以上信息，请回答下面的问题：

Q: 你们的退货政策是怎么样的？
"""
response, history = model.chat(tokenizer, question, history=[])
print(response)

A: 您的退货政策是在自收到商品之日起 7 天内，如产品未使用、包装完好，您可以申请退货。某些特殊商品可能不支持退货，请在购买前查看商品详情页面的退货政策。


In [7]:
question = """
Q: 你们的退货政策是怎么样的？
A:
"""
response, history = model.chat(tokenizer, question, history=[])
print(response)

我们的退货政策是在商品购买后的14天内。如果对购买的商品不满意，可以在退货政策规定的时间内退货。

具体的退货政策如下：

1. 退货时间：购买商品后的14天内。

2. 退货方式：可以在退货政策规定的退货地址退货，也可以使用在线退货服务。

3. 退货商品：退货时需要提供购买商品时的发票和订单号，以及退货原因。

4. 退货流程：如果使用在线退货服务，可以访问我们的退货服务网站，按照网站的指示完成退货流程。如果使用邮寄退货服务，可以将商品邮寄到指定的退货地址，我们会在收到退货后尽快处理。

5. 退款：退货完成后，我们会在收到退货后的2-3天内处理退款，并退回相应的退款金额。

请注意，退货政策可能会因地区、销售点、购买商品类型等因素而有所不同，如果有任何疑问，请查看我们的退货政策或与当地的客户服务联系。


In [8]:
question = """
我们支持全国大部分省份的配送，包括北京、上海、天津、重庆、河北、山西、辽宁、吉林、黑龙江、江苏、浙江、安徽、福建、江西、山东、河南、湖北、湖南、广东、海南、四川、贵州、云南、陕西、甘肃、青海、台湾、内蒙古、广西、西藏、宁夏和新疆.

根据以上信息，请回答下面的问题：

Q: 你们能配送到三亚吗？
"""
response, history = model.chat(tokenizer, question, history=[])
print(response)


根据提供的信息，我们支持全国大部分省份的配送，包括北京、上海、天津、重庆、河北、山西、辽宁、吉林、黑龙江、江苏、浙江、安徽、福建、江西、山东、河南、湖北、湖南、广东、海南、四川、贵州、云南、陕西、甘肃、青海、台湾、内蒙古、广西、西藏、宁夏和新疆.因此三亚也应该支持配送.


In [10]:
question = """
我们支持全国大部分省份的配送，包括北京、上海、天津、重庆、河北、山西、江苏、浙江、安徽、福建、江西、山东、河南、湖北、湖南、广东、海南、四川、贵州、云南、陕西、甘肃、青海、台湾、内蒙古、广西、西藏、宁夏和新疆.但是不能配送黑龙江

根据以上信息，请回答下面的问题：

Q: 你们能配送到哈尔滨吗？
"""
response, history = model.chat(tokenizer, question, history=[])
print(response)

根据提供的信息，我们支持全国大部分省份的配送，包括北京、上海、天津、重庆、河北、山西、江苏、浙江、安徽、福建、江西、山东、河南、湖北、湖南、广东、海南、四川、贵州、云南、陕西、甘肃、青海、台湾、内蒙古、广西、西藏、宁夏和新疆.但是不能配送黑龙江。因此，黑龙江省不能通过我们的服务进行配送。
